In [2]:
%pip install polars

Note: you may need to restart the kernel to use updated packages.


In [3]:
import polars as pl

In [4]:
df = pl.scan_csv(r'C:\Users\Anuj Bohra\Desktop\labevents_mimic4.csv')
df.schema

C:\Users\Anuj Bohra\AppData\Local\Temp\ipykernel_31544\3518913959.py:2: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  df.schema


Schema([('labevent_id', Int64),
        ('subject_id', Int64),
        ('hadm_id', String),
        ('specimen_id', Int64),
        ('itemid', Int64),
        ('charttime', String),
        ('storetime', String),
        ('value', String),
        ('valuenum', Float64),
        ('valueuom', String),
        ('ref_range_lower', Float64),
        ('ref_range_upper', Float64),
        ('flag', String),
        ('priority', String),
        ('comments', String)])

In [5]:
import polars as pl

# Get total row count efficiently
total_rows = df.select(pl.count()).collect().item()

# Calculate 8% of total rows (rounded to integer)
n = int(total_rows * 0.08)

# Keep first n rows using lazy execution
sampled_df = (
    df
    .slice(0, n)  # Equivalent to `.head(n)`
    .collect()     # Execute the query
)

# Export to CSV (replace with your output path)
sampled_df.write_csv("first_8_percent.csv")

C:\Users\Anuj Bohra\AppData\Local\Temp\ipykernel_31544\4277532658.py:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  total_rows = df.select(pl.count()).collect().item()


In [6]:
sampled_df = (
    df
    .slice(0, 2500000)  # Equivalent to `.head(n)`
    .collect()     # Execute the query
)

# Export to CSV (replace with your output path)
sampled_df.write_csv("2_5mil.csv")

In [7]:
import pandas as pd

In [8]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Anuj Bohra\Desktop\Arogo\Severity\2_5mil.csv')

In [9]:
df.head()

,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,1,10000032,NaN,45421181,51237,2180-03-23 11:51:00,2180-03-23 15:15:00,1.4,1.4,NaN,0.9,1.1,abnormal,ROUTINE,NaN
1,2,10000032,NaN,45421181,51274,2180-03-23 11:51:00,2180-03-23 15:15:00,___,15.1,sec,9.4,12.5,abnormal,ROUTINE,VERIFIED.
2,3,10000032,NaN,52958335,50853,2180-03-23 11:51:00,2180-03-25 11:06:00,___,15.0,ng/mL,30.0,60.0,abnormal,ROUTINE,NEW ASSAY IN USE ___: DETECTS D2 AND D3 25-OH ...
3,4,10000032,NaN,52958335,50861,2180-03-23 11:51:00,2180-03-23 16:40:00,102,102.0,IU/L,0.0,40.0,abnormal,ROUTINE,NaN
4,5,10000032,NaN,52958335,50862,2180-03-23 11:51:00,2180-03-23 16:40:00,3.3,3.3,g/dL,3.5,5.2,abnormal,ROUTINE,NaN


In [10]:
df = df[['subject_id', 'hadm_id', 'itemid', 'valuenum']]
df.head

<bound method NDFrame.head of          subject_id     hadm_id  itemid  valuenum
0          10000032         NaN   51237      1.40
1          10000032         NaN   51274     15.10
2          10000032         NaN   50853     15.00
3          10000032         NaN   50861    102.00
4          10000032         NaN   50862      3.30
...             ...         ...     ...       ...
2499995    10220107  23406923.0   51265    176.00
2499996    10220107  23406923.0   51277     17.50
2499997    10220107  23406923.0   51279      3.12
2499998    10220107  23406923.0   51301      5.20
2499999    10220107  23406923.0   50868     14.00

[2500000 rows x 4 columns]>

In [11]:
pivot_df = df.pivot_table(
    index=['subject_id', 'hadm_id'],  # Group by these identifiers
    columns='itemid',                 # Convert unique itemids into columns
    values='valuenum',                # Populate columns with valuenum
    aggfunc='first'                   # Handle duplicates (use 'first', 'mean', etc.)
).reset_index()                       # Flatten the index

pivot_df.columns.name = None

pivot_df

,subject_id,hadm_id,50801,50802,50803,50804,50805,50806,50808,50809,...,52276,52278,52281,52285,52286,52312,52358,52369,52391,52769
0,10000032,22595853.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000032,22841357.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000032,25742920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,29079034.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000084,23052089.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7269,10220107,23023366.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7270,10220107,23406923.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7271,10220107,25349952.0,NaN,4.0,NaN,28.0,NaN,106.0,1.12,110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7272,10220107,27122498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
na = pivot_df.isnull().sum()

d = dict(na)

for i in d:
    if (d[i]/7274) > 0.5:
        pivot_df.drop(i, axis=1, inplace=True)

pivot_df

,subject_id,hadm_id,50868,50882,50893,50902,50912,50931,50960,50970,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
0,10000032,22595853.0,9.0,28.0,7.8,105.0,0.3,99.0,1.7,3.6,...,1.5,33.4,33.8,99.0,71.0,16.6,32.3,15.3,3.80,4.2
1,10000032,22841357.0,14.0,25.0,7.8,92.0,0.3,71.0,1.9,3.3,...,1.5,34.5,35.0,99.0,137.0,16.3,NaN,15.7,3.60,6.6
2,10000032,25742920.0,11.0,26.0,8.6,95.0,0.6,95.0,2.4,3.9,...,1.5,35.9,34.8,103.0,133.0,15.8,32.5,16.0,3.36,7.5
3,10000032,29079034.0,14.0,21.0,9.3,102.0,0.5,115.0,2.3,2.4,...,1.8,34.9,34.1,102.0,94.0,19.5,34.2,15.8,3.40,4.1
4,10000084,23052089.0,15.0,19.0,9.9,98.0,0.8,100.0,2.0,3.7,...,1.0,31.8,33.5,95.0,263.0,11.1,23.4,13.0,4.03,8.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7269,10220107,23023366.0,12.0,27.0,8.8,104.0,1.2,142.0,2.1,2.8,...,3.3,29.6,32.3,92.0,185.0,34.9,57.1,14.8,3.65,3.5
7270,10220107,23406923.0,17.0,24.0,9.0,105.0,2.1,163.0,2.3,4.4,...,2.3,29.9,32.9,91.0,183.0,24.9,41.3,18.0,3.37,4.6
7271,10220107,25349952.0,15.0,25.0,8.7,104.0,1.2,104.0,2.0,3.5,...,1.0,29.1,34.5,85.0,258.0,10.8,70.8,13.5,4.27,6.4
7272,10220107,27122498.0,17.0,22.0,8.6,106.0,1.1,140.0,1.9,2.9,...,1.1,29.8,32.4,92.0,228.0,11.6,33.2,15.5,2.82,5.2


In [13]:
pivot_df.fillna(pivot_df.mean(numeric_only=True), inplace=True)
pivot_df

,subject_id,hadm_id,50868,50882,50893,50902,50912,50931,50960,50970,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
0,10000032,22595853.0,9.0,28.0,7.8,105.0,0.3,99.0,1.7,3.6,...,1.5,33.4,33.8,99.0,71.0,16.6,32.300000,15.3,3.80,4.2
1,10000032,22841357.0,14.0,25.0,7.8,92.0,0.3,71.0,1.9,3.3,...,1.5,34.5,35.0,99.0,137.0,16.3,36.346258,15.7,3.60,6.6
2,10000032,25742920.0,11.0,26.0,8.6,95.0,0.6,95.0,2.4,3.9,...,1.5,35.9,34.8,103.0,133.0,15.8,32.500000,16.0,3.36,7.5
3,10000032,29079034.0,14.0,21.0,9.3,102.0,0.5,115.0,2.3,2.4,...,1.8,34.9,34.1,102.0,94.0,19.5,34.200000,15.8,3.40,4.1
4,10000084,23052089.0,15.0,19.0,9.9,98.0,0.8,100.0,2.0,3.7,...,1.0,31.8,33.5,95.0,263.0,11.1,23.400000,13.0,4.03,8.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7269,10220107,23023366.0,12.0,27.0,8.8,104.0,1.2,142.0,2.1,2.8,...,3.3,29.6,32.3,92.0,185.0,34.9,57.100000,14.8,3.65,3.5
7270,10220107,23406923.0,17.0,24.0,9.0,105.0,2.1,163.0,2.3,4.4,...,2.3,29.9,32.9,91.0,183.0,24.9,41.300000,18.0,3.37,4.6
7271,10220107,25349952.0,15.0,25.0,8.7,104.0,1.2,104.0,2.0,3.5,...,1.0,29.1,34.5,85.0,258.0,10.8,70.800000,13.5,4.27,6.4
7272,10220107,27122498.0,17.0,22.0,8.6,106.0,1.1,140.0,1.9,2.9,...,1.1,29.8,32.4,92.0,228.0,11.6,33.200000,15.5,2.82,5.2


In [17]:
diag = pd.read_csv(r'C:\Users\Anuj Bohra\Desktop\Arogo\Severity\patients-and-icdcode\diagnoses_icd (1).csv')
diag

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
...,...,...,...,...,...
4752260,19999987,23865745,7,41401,9
4752261,19999987,23865745,8,78039,9
4752262,19999987,23865745,9,0413,9
4752263,19999987,23865745,10,36846,9


In [18]:
diag = diag[diag['seq_num'] == 1]
diag = diag[['hadm_id', 'icd_code', 'icd_version']]
diag

,hadm_id,icd_code,icd_version
0,22595853,5723,9
8,22841357,07071,9
16,25742920,07054,9
26,29079034,45829,9
39,25022803,30500,9
...,...,...,...
4752194,25744818,T8141XA,10
4752213,29734428,T8131XA,10
4752235,21033226,3453,9
4752247,26071774,43491,9


In [19]:
admissions = pd.read_csv(r'C:\Users\Anuj Bohra\Desktop\Arogo\Severity\patients-and-icdcode\admissions.csv')
admissions = admissions[['subject_id', 'hadm_id']]
admissions

,subject_id,hadm_id
0,10000032,22595853
1,10000032,22841357
2,10000032,25742920
3,10000032,29079034
4,10000068,25022803
...,...,...
431083,19999828,25744818
431084,19999828,29734428
431085,19999840,21033226
431086,19999840,26071774


In [20]:
gender = pd.read_csv(r'C:\Users\Anuj Bohra\Desktop\Arogo\Severity\patients-and-icdcode\patients.csv')
gender = gender[['subject_id', 'gender']]
gender

,subject_id,gender
0,10000032,F
1,10000048,F
2,10000068,F
3,10000084,M
4,10000102,F
...,...,...
299772,19999828,F
299773,19999829,F
299774,19999840,M
299775,19999914,F


In [21]:
final_df = pd.merge(
    gender, 
    admissions, 
    on='subject_id',  
    how='left'        
)

final_df

,subject_id,gender,hadm_id
0,10000032,F,22595853.0
1,10000032,F,22841357.0
2,10000032,F,25742920.0
3,10000032,F,29079034.0
4,10000048,F,NaN
...,...,...,...
550113,19999829,F,NaN
550114,19999840,M,21033226.0
550115,19999840,M,26071774.0
550116,19999914,F,NaN


In [22]:
diag

,hadm_id,icd_code,icd_version
0,22595853,5723,9
8,22841357,07071,9
16,25742920,07054,9
26,29079034,45829,9
39,25022803,30500,9
...,...,...,...
4752194,25744818,T8141XA,10
4752213,29734428,T8131XA,10
4752235,21033226,3453,9
4752247,26071774,43491,9


In [23]:
final_df = pd.merge(
    final_df, 
    diag, 
    on='hadm_id', 
    how='left'     
)

final_df

,subject_id,gender,hadm_id,icd_code,icd_version
0,10000032,F,22595853.0,5723,9.0
1,10000032,F,22841357.0,07071,9.0
2,10000032,F,25742920.0,07054,9.0
3,10000032,F,29079034.0,45829,9.0
4,10000048,F,NaN,NaN,NaN
...,...,...,...,...,...
550122,19999829,F,NaN,NaN,NaN
550123,19999840,M,21033226.0,3453,9.0
550124,19999840,M,26071774.0,43491,9.0
550125,19999914,F,NaN,NaN,NaN


In [24]:
pivot_df.drop('subject_id', axis=1, inplace=True)

In [25]:
final_df = pd.merge(
    final_df, 
    pivot_df, 
    on='hadm_id',  
    how='left'     
)

final_df

,subject_id,gender,hadm_id,icd_code,icd_version,50868,50882,50893,50902,50912,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
0,10000032,F,22595853.0,5723,9.0,9.0,28.0,7.8,105.0,0.3,...,1.5,33.4,33.8,99.0,71.0,16.6,32.300000,15.3,3.80,4.2
1,10000032,F,22841357.0,07071,9.0,14.0,25.0,7.8,92.0,0.3,...,1.5,34.5,35.0,99.0,137.0,16.3,36.346258,15.7,3.60,6.6
2,10000032,F,25742920.0,07054,9.0,11.0,26.0,8.6,95.0,0.6,...,1.5,35.9,34.8,103.0,133.0,15.8,32.500000,16.0,3.36,7.5
3,10000032,F,29079034.0,45829,9.0,14.0,21.0,9.3,102.0,0.5,...,1.8,34.9,34.1,102.0,94.0,19.5,34.200000,15.8,3.40,4.1
4,10000048,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550122,19999829,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550123,19999840,M,21033226.0,3453,9.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550124,19999840,M,26071774.0,43491,9.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550125,19999914,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
final_df = final_df.dropna()

In [27]:
final_df.to_csv('combined_cleaned_df_final.csv')

In [28]:
final_df

,subject_id,gender,hadm_id,icd_code,icd_version,50868,50882,50893,50902,50912,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
0,10000032,F,22595853.0,5723,9.0,9.0,28.0,7.8,105.0,0.3,...,1.5,33.4,33.8,99.0,71.0,16.6,32.300000,15.3,3.80,4.2
1,10000032,F,22841357.0,07071,9.0,14.0,25.0,7.8,92.0,0.3,...,1.5,34.5,35.0,99.0,137.0,16.3,36.346258,15.7,3.60,6.6
2,10000032,F,25742920.0,07054,9.0,11.0,26.0,8.6,95.0,0.6,...,1.5,35.9,34.8,103.0,133.0,15.8,32.500000,16.0,3.36,7.5
3,10000032,F,29079034.0,45829,9.0,14.0,21.0,9.3,102.0,0.5,...,1.8,34.9,34.1,102.0,94.0,19.5,34.200000,15.8,3.40,4.1
6,10000084,M,23052089.0,G3183,10.0,15.0,19.0,9.9,98.0,0.8,...,1.0,31.8,33.5,95.0,263.0,11.1,23.400000,13.0,4.03,8.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11641,10220107,M,23023366.0,5779,9.0,12.0,27.0,8.8,104.0,1.2,...,3.3,29.6,32.3,92.0,185.0,34.9,57.100000,14.8,3.65,3.5
11642,10220107,M,23406923.0,42823,9.0,17.0,24.0,9.0,105.0,2.1,...,2.3,29.9,32.9,91.0,183.0,24.9,41.300000,18.0,3.37,4.6
11644,10220107,M,25349952.0,41401,9.0,15.0,25.0,8.7,104.0,1.2,...,1.0,29.1,34.5,85.0,258.0,10.8,70.800000,13.5,4.27,6.4
11645,10220107,M,27122498.0,99811,9.0,17.0,22.0,8.6,106.0,1.1,...,1.1,29.8,32.4,92.0,228.0,11.6,33.200000,15.5,2.82,5.2
